In [23]:
import pandas as pd
import pickle

import gurobipy as gp
from gurobipy import GRB, quicksum

In [5]:
w = pd.read_csv('../data/terminal-and-depots.csv')
w.loc[w.type=='terminal', 'id'] = range(len(w[w.type=='terminal']))
w.head(2)

,line,lng,lat,type,id
0,35路,103.976832,30.688767,depot,0
1,43路,104.063147,30.656359,depot,1


In [9]:
terminal_df = w[w.type=='terminal']
depot_df = w[w.type=='depot']

terminals = [r.id for i, r in terminal_df.iterrows()]
depots = [r.id for i, r in depot_df.iterrows()]

depots

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [20]:
# 获取d_ij
with open('../data/parameter-dij.pkl', 'rb') as f:  # write binary
    d = pickle.load(f)
d[0][0:5]

[17355.71904935039,
 3892.3573955943707,
 18029.849691592917,
 3750.281401231461,
 14423.896337114571]

In [25]:
M = 5
Nj = 2

# Gurobi建模

In [26]:
m = gp.Model()

m.Params.Timelimit = 3600
m.Params.Gap = 0

x = m.addVars(depots, vtype=GRB.BINARY, name='x')
y = m.addVars([(i,j) for i in depots for j in terminals], vtype=GRB.BINARY, name='y')
z = m.addVars([(i,j) for i in depots for j in terminals], vtype=GRB.BINARY, name='z')

m.addConstrs((z.sum('*', j)==1 for j in terminals))
m.addConstr(x.sum()==M)
m.addConstrs((z[i,j]<=x[i] for i in depots for j in terminals))
m.addConstrs((z[i,j]<=y[i,j] for i in depots for j in terminals))
m.addConstrs((z[i,j]>=x[i]+y[i,j]-1 for i in depots for j in terminals))

m.setObjective(2*quicksum([z[i,j]*d[i][j]*Nj for i in depots for j in terminals]),
              sense=GRB.MINIMIZE)

m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-26
Set parameter TimeLimit to value 3600
No parameters matching 'Gap' found
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 15129 rows, 9940 columns and 39700 nonzeros
Model fingerprint: 0x208f19a3
Variable types: 0 continuous, 9940 integer (9940 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 1.329409e+07
Presolve removed 9920 rows and 4960 columns
Presolve time: 0.04s
Presolved: 5209 rows, 4980 columns, 14900 nonzeros
Variable types: 0 continuous, 4980 integer (4980 binary)
Found heuristic solution: objective 8315704.1033

Root relaxation: objective 4.157040e+06, 1316 iterations, 0.03 seconds (0.04 work units)

    Nodes    |    Current No

In [28]:
m.ObjVal / 1000.0

4157.039833469444

# 最优结果提取

In [35]:
selected_depots = [(i, m.getVarByName('x[{}]'.format(i)).x) for i in depots]
selected_depots = [i for i in selected_depots if i[1]>0]
selected_depots

[(0, 1.0), (1, 1.0), (2, 1.0), (14, 1.0), (19, 1.0)]

In [39]:
terminal_assign = [(i, j, m.getVarByName('y[{},{}]'.format(i,j)).x) for i in depots for j in terminals]
terminal_assign = [i for i in terminal_assign if i[2]>0]
terminal_assign[0:5]

[(0, 1, 1.0), (0, 3, 1.0), (0, 14, 1.0), (0, 17, 1.0), (0, 23, 1.0)]

# 作业，将优化结果可视化